In [ ]:
!pip uninstall -y numpy
!pip install --force-reinstall numpy==1.26.4

!pip install torch==2.2.1 torchvision==0.17.1 torchaudio==2.2.1 sentence-transformers==2.2.2
!pip install pandas==2.0.0
!pip install sparqlwrapper==2.0.0 transformers==4.41.0 requests==2.31.0 scikit-learn==1.2.0 pyrdf2vec
!pip install huggingface-hub==0.25.2
!pip install pyRDF2vec
!pip install rdflib==6.0.2
!pip install -U gensim
!pip install -U datasets
!pip install nlpaug
!pip install nltk==3.8.1 textstat==0.7.4 fuzzywuzzy==0.18.0 python-Levenshtein==0.25.1
!pip install rouge-score==0.1.2 networkx==3.1 umap-learn==0.5.5 matplotlib-venn==0.11.10
!pip install tqdm==4.66.5 -q
!pip install bert-score==0.3.13 -q

Found existing installation: numpy 2.0.2
Uninstalling numpy-2.0.2:
  Successfully uninstalled numpy-2.0.2
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.0/61.0 kB 1.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.3/18.3 MB 72.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.0/86.0 kB 4.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 755.6/755.6 MB 1.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 106.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 102.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 410.6/410.6 MB 3.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.1/14.1 MB 107.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.7/23.7 MB 92.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 823.6/823.6 kB 57.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 731.7/731.7 MB

In [ ]:
# Setup and Imports

import pandas as pd
import numpy as np
import torch
from torch.utils.data import Dataset, DataLoader
from transformers import (
    T5ForConditionalGeneration, T5Tokenizer,
    DPRContextEncoder, DPRQuestionEncoder,
    DPRContextEncoderTokenizer, DPRQuestionEncoderTokenizer,
    BertModel, BertTokenizer
)
from sentence_transformers import SentenceTransformer
from sklearn.model_selection import train_test_split
import os
from google.colab import drive
from tqdm import tqdm
import matplotlib.pyplot as plt
import seaborn as sns
from nltk.translate.bleu_score import sentence_bleu
from rouge_score import rouge_scorer
from bert_score import score as bert_score
from torch.cuda.amp import GradScaler, autocast
from torch.optim.lr_scheduler import CosineAnnealingLR
from nltk.corpus import wordnet
import nltk
import json
import string

nltk.download('wordnet')
nltk.download('punkt')

drive.mount('/content/drive')

# Updated Config class
class Config:
    BASE_PATH = "/content/drive/MyDrive/LJMU-Datasets"
    DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    BERT_MODEL_NAME = "bert-base-uncased"
    T5_MODEL_NAME = "t5-small"
    DPR_CTX_MODEL_NAME = "facebook/dpr-ctx_encoder-single-nq-base"
    DPR_QUESTION_MODEL_NAME = "facebook/dpr-question_encoder-single-nq-base"
    BATCH_SIZE = 16  # Adjusted for DPR fine-tuning
    MAX_EPOCHS = 3  # For DPR and T5 fine-tuning
    NUM_WORKERS = 4
    MAX_LENGTH = 256
    SUBSET_SIZE = 500
    HOTPOTQA_MAX_SAMPLES = 1000
    WIKIDATA_SUBSET_SIZE = 30000

CONFIG = Config()
print(f"Using device: {CONFIG.DEVICE}")

# Clear GPU memory
torch.cuda.empty_cache()

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


Mounted at /content/drive
Using device: cuda


In [ ]:
# Data Collection and Preprocessing

# Load datasets
qa_train_path = os.path.join(CONFIG.BASE_PATH, "qa_train.csv")
qa_val_path = os.path.join(CONFIG.BASE_PATH, "qa_val.csv")
triple_train_path = os.path.join(CONFIG.BASE_PATH, "triple_train.csv")

qa_train_df = pd.read_csv(qa_train_path)
qa_val_df = pd.read_csv(qa_val_path)
triple_train_df = pd.read_csv(triple_train_path)

# Balance datasets
min_size = min(len(qa_train_df), len(triple_train_df))
qa_train_df = qa_train_df.sample(n=min_size, random_state=42)
triple_train_df = triple_train_df.sample(n=min_size, random_state=42)

print(f"Balanced datasets: QA Train={len(qa_train_df)}, QA Val={len(qa_val_df)}, Triple Train={len(triple_train_df)}")

# Custom Dataset for T5 and DPR
class RetrievalDataset(Dataset):
    def __init__(self, df: pd.DataFrame, t5_tokenizer: T5Tokenizer, dpr_question_tokenizer: DPRQuestionEncoderTokenizer,
                 max_length: int = 256, task: str = "qa", candidate_objects: list = None):
        self.t5_tokenizer = t5_tokenizer
        self.dpr_question_tokenizer = dpr_question_tokenizer
        self.max_length = max_length
        self.task = task
        self.data = df
        self.candidate_objects = candidate_objects

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        row = self.data.iloc[idx]
        question = row["question"]
        context = row["context"]
        answer = row["answer"]

        if self.task == "qa":
            t5_input_text = f"answer the question exactly as in the reference: {question} context: {context} ->"
        else:
            t5_input_text = f"complete the triple with the exact object: {question} ->"
        t5_inputs = self.t5_tokenizer(
            t5_input_text,
            return_tensors="pt",
            max_length=self.max_length,
            truncation=True,
            padding="max_length"
        )
        t5_labels = self.t5_tokenizer(
            answer,
            return_tensors="pt",
            max_length=self.max_length,
            truncation=True,
            padding="max_length"
        )

        dpr_inputs = self.dpr_question_tokenizer(
            question,
            return_tensors="pt",
            max_length=self.max_length,
            truncation=True,
            padding="max_length"
        )

        item = {
            "task": self.task,
            "t5_input_ids": t5_inputs["input_ids"].squeeze(),
            "t5_attention_mask": t5_inputs["attention_mask"].squeeze(),
            "t5_labels": t5_labels["input_ids"].squeeze(),
            "dpr_input_ids": dpr_inputs["input_ids"].squeeze(),
            "dpr_attention_mask": dpr_inputs["attention_mask"].squeeze(),
            "question": question,
            "answer": answer
        }

        if self.task == "triple" and self.candidate_objects:
            label_idx = self.candidate_objects.index(answer) if answer in self.candidate_objects else -1
            item["label_idx"] = label_idx

        return item

t5_tokenizer = T5Tokenizer.from_pretrained(CONFIG.T5_MODEL_NAME)
dpr_question_tokenizer = DPRQuestionEncoderTokenizer.from_pretrained(CONFIG.DPR_QUESTION_MODEL_NAME)

triple_candidates = list(set(triple_train_df["answer"].tolist()))
print(f"Number of unique triple candidates: {len(triple_candidates)}")

qa_train_dataset = RetrievalDataset(qa_train_df, t5_tokenizer, dpr_question_tokenizer, task="qa")
qa_val_dataset = RetrievalDataset(qa_val_df, t5_tokenizer, dpr_question_tokenizer, task="qa")
triple_train_dataset = RetrievalDataset(triple_train_df, t5_tokenizer, dpr_question_tokenizer, task="triple", candidate_objects=triple_candidates)

qa_train_loader = DataLoader(qa_train_dataset, batch_size=CONFIG.BATCH_SIZE, shuffle=True, num_workers=CONFIG.NUM_WORKERS)
qa_val_loader = DataLoader(qa_val_dataset, batch_size=CONFIG.BATCH_SIZE, shuffle=False, num_workers=CONFIG.NUM_WORKERS)
triple_train_loader = DataLoader(triple_train_dataset, batch_size=CONFIG.BATCH_SIZE, shuffle=True, num_workers=CONFIG.NUM_WORKERS)

print(f"Created DataLoaders: QA Train={len(qa_train_dataset)}, QA Val={len(qa_val_dataset)}, Triple Train={len(triple_train_dataset)}")

Balanced datasets: QA Train=1400, QA Val=100, Triple Train=1400


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_token.py:90: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/2.32k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.39M [00:00<?, ?B/s]

...

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

/usr/local/lib/python3.11/dist-packages/huggingface_hub/file_download.py:1142: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


config.json:   0%|          | 0.00/493 [00:00<?, ?B/s]

Number of unique triple candidates: 850
Created DataLoaders: QA Train=1400, QA Val=100, Triple Train=1400


In [ ]:
# Define Custom Loss Functions

import numpy as np
import torch

# Hard negative selection (for DPR embeddings)
def select_hard_negatives(embeddings, candidate_embeddings, correct_answers, all_candidates, num_negatives: int = 10):
    embeddings = embeddings.cpu().detach().numpy()
    candidate_embeddings = candidate_embeddings.cpu().detach().numpy()
    similarities = np.dot(embeddings, candidate_embeddings.T)
    hard_negative_indices = []
    for i in range(len(embeddings)):
        correct_idx = all_candidates.index(correct_answers[i]) if correct_answers[i] in all_candidates else -1
        sim_scores = similarities[i].copy()
        if correct_idx != -1:
            sim_scores[correct_idx] = -float('inf')  # Exclude correct answer
        valid_indices = np.where(sim_scores > 0.8)[0]
        if len(valid_indices) < num_negatives:
            valid_indices = np.argsort(-sim_scores)[:num_negatives]
        else:
            valid_indices = np.argsort(-sim_scores[valid_indices])[:num_negatives]
        hard_negative_indices.append(valid_indices)
    return torch.tensor(hard_negative_indices, device=CONFIG.DEVICE)

# InfoNCE loss for DPR fine-tuning
def info_nce_loss(similarities, labels):
    return torch.nn.functional.cross_entropy(similarities, labels)

# T5 generation loss (for compatibility)
def t5_generation_loss(outputs, labels, ignore_index: int = t5_tokenizer.pad_token_id):
    logits = outputs.logits
    shift_logits = logits[..., :-1, :].contiguous()
    shift_labels = labels[..., 1:].contiguous()
    loss_fn = torch.nn.CrossEntropyLoss(ignore_index=ignore_index)
    loss = loss_fn(shift_logits.view(-1, shift_logits.size(-1)), shift_labels.view(-1))
    return loss

# T5 classification loss (for compatibility)
def t5_classification_loss(outputs, labels):
    logits = outputs.logits
    loss_fn = torch.nn.CrossEntropyLoss(ignore_index=-1)
    loss = loss_fn(logits, labels)
    return loss

# Normalize text for evaluation
def normalize_text(text: str) -> str:
    text = str(text).lower().strip()
    text = text.translate(str.maketrans("", "", string.punctuation))
    articles = {'a', 'an', 'the'}
    words = text.split()
    words = [word for word in words if word not in articles]
    return ' '.join(words)

print("Defined custom loss functions for T5 and DPR.")

Defined custom loss functions for T5 and DPR.


In [ ]:
# Fine-Tune T5-small for QA with A100 Optimizations

import torch.optim as optim

def fine_tune_t5_qa(train_loader, val_loader, epochs: int = 3, checkpoint_path: str = None):
    print("Fine-tuning T5-small for QA...")

    t5_model = T5ForConditionalGeneration.from_pretrained(CONFIG.T5_MODEL_NAME).to(CONFIG.DEVICE)
    optimizer = optim.AdamW(t5_model.parameters(), lr=3e-4)
    total_steps = len(train_loader) * epochs
    warmup_steps = int(total_steps * 0.1)
    scheduler = CosineAnnealingLR(optimizer, T_max=total_steps - warmup_steps, eta_min=1e-6)
    scaler = GradScaler()

    start_epoch = 0
    best_loss = float("inf")
    patience, max_patience = 0, 5

    if checkpoint_path and os.path.exists(checkpoint_path):
        checkpoint = torch.load(checkpoint_path, map_location=CONFIG.DEVICE)
        t5_model.load_state_dict(checkpoint["model_state_dict"])
        optimizer.load_state_dict(checkpoint["optimizer_state_dict"])
        scheduler.load_state_dict(checkpoint["scheduler_state_dict"])
        start_epoch = checkpoint["epoch"] + 1
        best_loss = checkpoint["best_loss"]
        patience = checkpoint["patience"]
        print(f"Resumed training from checkpoint at epoch {start_epoch} with best loss {best_loss:.4f}")

    for epoch in range(start_epoch, epochs):
        t5_model.train()
        total_loss = 0
        for step, batch in enumerate(tqdm(train_loader, desc=f"Epoch {epoch+1}/{epochs}")):
            input_ids = batch["t5_input_ids"].to(CONFIG.DEVICE)
            attention_mask = batch["t5_attention_mask"].to(CONFIG.DEVICE)
            labels = batch["t5_labels"].to(CONFIG.DEVICE)

            optimizer.zero_grad()
            with autocast():
                outputs = t5_model(input_ids=input_ids, attention_mask=attention_mask, labels=labels)
                loss = t5_generation_loss(outputs, labels)
            scaler.scale(loss).backward()
            scaler.step(optimizer)
            scaler.update()

            total_loss += loss.item()

            if step < warmup_steps:
                lr = (step + 1) / warmup_steps * 3e-4
                for param_group in optimizer.param_groups:
                    param_group['lr'] = lr
            else:
                scheduler.step()

            del input_ids, attention_mask, labels, outputs, loss
            torch.cuda.empty_cache()

        avg_loss = total_loss / len(train_loader)
        print(f"Epoch {epoch+1}/{epochs} - Train Loss: {avg_loss:.4f}, LR: {optimizer.param_groups[0]['lr']}")

        t5_model.eval()
        val_loss = 0
        with torch.no_grad():
            for batch in val_loader:
                input_ids = batch["t5_input_ids"].to(CONFIG.DEVICE)
                attention_mask = batch["t5_attention_mask"].to(CONFIG.DEVICE)
                labels = batch["t5_labels"].to(CONFIG.DEVICE)
                with autocast():
                    outputs = t5_model(input_ids=input_ids, attention_mask=attention_mask, labels=labels)
                    val_loss += t5_generation_loss(outputs, labels).item()
                del input_ids, attention_mask, labels, outputs
                torch.cuda.empty_cache()
        val_loss /= len(val_loader)
        print(f"Epoch {epoch+1}/{epochs} - Val Loss: {val_loss:.4f}")

        with torch.no_grad():
            batch = next(iter(val_loader))
            input_ids = batch["t5_input_ids"][:5].to(CONFIG.DEVICE)
            attention_mask = batch["t5_attention_mask"][:5].to(CONFIG.DEVICE)
            generated_ids = t5_model.generate(
                input_ids=input_ids,
                attention_mask=attention_mask,
                max_new_tokens=100,
                num_beams=20,
                temperature=0.5,
                no_repeat_ngram_size=2
            )
            generated_texts = [t5_tokenizer.decode(g_ids, skip_special_tokens=True).lower().strip() for g_ids in generated_ids]
            for gen, ref in zip(generated_texts, batch["answer"][:5]):
                print(f"Generated: {gen}")
                print(f"Reference: {ref}\n")
            del input_ids, attention_mask, generated_ids
            torch.cuda.empty_cache()

        checkpoint = {
            "epoch": epoch,
            "model_state_dict": t5_model.state_dict(),
            "optimizer_state_dict": optimizer.state_dict(),
            "scheduler_state_dict": scheduler.state_dict(),
            "train_loss": avg_loss,
            "val_loss": val_loss,
            "best_loss": best_loss,
            "patience": patience
        }
        epoch_checkpoint_path = os.path.join(CONFIG.BASE_PATH, f"t5_qa_checkpoint_epoch_{epoch+1}.pt")
        torch.save(checkpoint, epoch_checkpoint_path)
        print(f"Saved checkpoint for epoch {epoch+1} at {epoch_checkpoint_path}")

        if val_loss < best_loss:
            best_loss = val_loss
            patience = 0
            best_checkpoint_path = os.path.join(CONFIG.BASE_PATH, "t5_qa_best.pt")
            torch.save(checkpoint, best_checkpoint_path)
            print(f"Saved best T5 QA model with val loss {best_loss:.4f} at {best_checkpoint_path}")
        else:
            patience += 1
            if patience >= max_patience:
                print("Early stopping triggered.")
                break

    return t5_model

try:
    t5_qa_model = fine_tune_t5_qa(qa_train_loader, qa_val_loader, checkpoint_path=None)
except Exception as e:
    print(f"Error in T5 QA fine-tuning: {e}")
    raise

Fine-tuning T5-small for QA...


Epoch 1/3: 100%|██████████| 88/88 [00:12<00:00,  7.08it/s]

Epoch 1/3 - Train Loss: 6.0765, LR: 0.0002526670388219687


Epoch 1/3 - Val Loss: 3.1410


/usr/local/lib/python3.11/dist-packages/transformers/generation/configuration_utils.py:515: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.5` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(


...

Epoch 2/3 - Train Loss: 4.5978, LR: 0.00016690339156863067


Epoch 2/3 - Val Loss: 2.4310
Generated: 80%
Reference: 80%

Generated: ansans
Reference: unanswerable

Generated: ansans
Reference: external senses

Generated: ington college
Reference: wellington college

Generated: ansans
Reference: 200 million kilowatt hours

Saved checkpoint for epoch 2 at /content/drive/MyDrive/LJMU-Datasets/t5_qa_checkpoint_epoch_2.pt
Saved best T5 QA model with val loss 2.4310 at /content/drive/MyDrive/LJMU-Datasets/t5_qa_best.pt


Epoch 3/3: 100%|██████████| 88/88 [00:11<00:00,  7.54it/s]

Epoch 3/3 - Train Loss: 4.2609, LR: 7.349462576098299e-05


Epoch 3/3 - Val Loss: 2.1435
Generated: 80%
Reference: 80%

Generated: ansans
Reference: unanswerable

Generated: by our external senses this sensory information is supplied to internals which merge all pieces into whole unified conscious experience this process of perception and abstraction is nex of soul and body for material body may only only objects while immaterial soul may on ->
Reference: external senses

Generated: ington
Reference: wellington college

Generated: 200 million watt
Reference: 200 million kilowatt hours

Saved checkpoint for epoch 3 at /content/drive/MyDrive/LJMU-Datasets/t5_qa_checkpoint_epoch_3.pt
Saved best T5 QA model with val loss 2.1435 at /content/drive/MyDrive/LJMU-Datasets/t5_qa_best.pt


In [ ]:
# Fine-Tune T5-small for Triple Retrieval with Classification Head

from transformers import T5ForConditionalGeneration
import torch.nn as nn
import torch
import logging
from transformers.modeling_outputs import Seq2SeqLMOutput

class T5ForTripleClassification(T5ForConditionalGeneration):
    def __init__(self, config, num_classes):
        super().__init__(config)
        self.classification_head = nn.Linear(config.d_model, num_classes)

    def forward(self, input_ids, attention_mask=None, decoder_input_ids=None, labels=None, label_idx=None):
        if label_idx is not None:
            encoder_outputs = self.encoder(input_ids=input_ids, attention_mask=attention_mask)
            encoder_hidden = encoder_outputs.last_hidden_state[:, 0, :]
            logits = self.classification_head(encoder_hidden)
            return Seq2SeqLMOutput(logits=logits)
        else:
            if decoder_input_ids is None and labels is not None:
                decoder_input_ids = self._shift_right(labels)
            if decoder_input_ids is None and labels is None:
                decoder_start = getattr(self.config, "decoder_start_token_id", self.config.pad_token_id)
                decoder_input_ids = torch.full((input_ids.size(0), 1), decoder_start, device=input_ids.device)
            outputs = super().forward(
                input_ids=input_ids,
                attention_mask=attention_mask,
                decoder_input_ids=decoder_input_ids,
                labels=labels
            )
            return outputs

def fine_tune_t5_triple(train_loader, val_loader, num_classes: int, epochs: int = CONFIG.MAX_EPOCHS, checkpoint_path: str = None):
    print("Fine-tuning T5-small for triple retrieval with classification head...")

    t5_model = T5ForTripleClassification.from_pretrained(CONFIG.T5_MODEL_NAME, num_classes=num_classes).to(CONFIG.DEVICE)
    t5_generator = T5ForConditionalGeneration.from_pretrained(CONFIG.T5_MODEL_NAME).to(CONFIG.DEVICE)
    optimizer = optim.AdamW(t5_model.parameters(), lr=3e-4)
    total_steps = len(train_loader) * epochs
    warmup_steps = int(total_steps * 0.1)
    scheduler = CosineAnnealingLR(optimizer, T_max=total_steps - warmup_steps, eta_min=1e-6)
    scaler = GradScaler()

    start_epoch = 0
    best_loss = float("inf")
    patience, max_patience = 0, 5

    if checkpoint_path and os.path.exists(checkpoint_path):
        checkpoint = torch.load(checkpoint_path, map_location=CONFIG.DEVICE)
        t5_model.load_state_dict(checkpoint["model_state_dict"])
        optimizer.load_state_dict(checkpoint["optimizer_state_dict"])
        scheduler.load_state_dict(checkpoint["scheduler_state_dict"])
        start_epoch = checkpoint["epoch"] + 1
        best_loss = checkpoint["best_loss"]
        patience = checkpoint["patience"]
        print(f"Resumed training from checkpoint at epoch {start_epoch} with best loss {best_loss:.4f}")

    for epoch in range(start_epoch, epochs):
        t5_model.train()
        total_loss = 0
        for step, batch in enumerate(tqdm(train_loader, desc=f"Epoch {epoch+1}/{epochs}")):
            input_ids = batch["t5_input_ids"].to(CONFIG.DEVICE)
            attention_mask = batch["t5_attention_mask"].to(CONFIG.DEVICE)
            label_idx = batch["label_idx"].to(CONFIG.DEVICE)

            optimizer.zero_grad()
            with autocast():
                outputs = t5_model(input_ids=input_ids, attention_mask=attention_mask, label_idx=label_idx)
                loss = t5_classification_loss(outputs, label_idx)
            scaler.scale(loss).backward()
            scaler.step(optimizer)
            scaler.update()

            total_loss += loss.item()

            if step < warmup_steps:
                lr = (step + 1) / warmup_steps * 3e-4
                for param_group in optimizer.param_groups:
                    param_group['lr'] = lr
            else:
                scheduler.step()

            del input_ids, attention_mask, label_idx, outputs, loss
            torch.cuda.empty_cache()

        avg_loss = total_loss / len(train_loader)
        print(f"Epoch {epoch+1}/{epochs} - Train Loss: {avg_loss:.4f}, LR: {optimizer.param_groups[0]['lr']}")

        t5_model.eval()
        val_loss = 0
        with torch.no_grad():
            for batch in train_loader:
                input_ids = batch["t5_input_ids"].to(CONFIG.DEVICE)
                attention_mask = batch["t5_attention_mask"].to(CONFIG.DEVICE)
                label_idx = batch["label_idx"].to(CONFIG.DEVICE)
                with autocast():
                    outputs = t5_model(input_ids=input_ids, attention_mask=attention_mask, label_idx=label_idx)
                    val_loss += t5_classification_loss(outputs, label_idx).item()
                del input_ids, attention_mask, label_idx, outputs
                torch.cuda.empty_cache()
        val_loss /= len(train_loader)
        print(f"Epoch {epoch+1}/{epochs} - Val Loss: {avg_loss:.4f}")

        with torch.no_grad():
            batch = next(iter(val_loader))
            input_ids = batch["t5_input_ids"][:5].to(CONFIG.DEVICE)
            attention_mask = batch["t5_attention_mask"][:5].to(CONFIG.DEVICE)
            generated_ids = t5_generator.generate(
                input_ids=input_ids,
                attention_mask=attention_mask,
                max_new_tokens=50,
                num_beams=15,
                temperature=0.5,
                no_repeat_ngram_size=2
            )
            generated_texts = [t5_tokenizer.decode(g_ids, skip_special_tokens=True).lower().strip() for g_ids in generated_ids]
            for gen, ref in zip(generated_texts, batch["answer"][:5]):
                print(f"Generated: {gen}")
                print(f"Reference: {ref}\n")
            del input_ids, attention_mask, generated_ids
            torch.cuda.empty_cache()

        checkpoint = {
            "epoch": epoch,
            "model_state_dict": t5_model.state_dict(),
            "optimizer_state_dict": optimizer.state_dict(),
            "scheduler_state_dict": scheduler.state_dict(),
            "train_loss": avg_loss,
            "val_loss": val_loss,
            "best_loss": best_loss,
            "patience": patience
        }
        epoch_checkpoint_path = os.path.join(CONFIG.BASE_PATH, f"t5_triple_checkpoint_epoch_{epoch+1}.pt")
        torch.save(checkpoint, epoch_checkpoint_path)
        print(f"Saved checkpoint for epoch {epoch+1} at {epoch_checkpoint_path}")

        if val_loss < best_loss:
            best_loss = val_loss
            patience = 0
            best_checkpoint_path = os.path.join(CONFIG.BASE_PATH, "t5_triple_best.pt")
            torch.save(checkpoint, best_checkpoint_path)
            print(f"Saved best T5 Triple model with val loss {best_loss:.4f} at {best_checkpoint_path}")
        else:
            patience += 1
            if patience >= max_patience:
                print("Early stopping triggered.")
                break

    return t5_model

# Skip fine-tuning since we're not using T5 for triple classification in the backup plan
print("Skipping T5 triple fine-tuning as we're using DPR for ranking.")

Skipping T5 triple fine-tuning as we're using DPR for ranking.


In [ ]:
# Save Artifacts to Google Drive

import pickle
import torch
import os
from google.colab import drive
from sentence_transformers import SentenceTransformer

# Mount Google Drive
drive.mount('/content/drive')

# Define save path
save_path = '/content/drive/MyDrive/bert_retrieval_artifacts'
os.makedirs(save_path, exist_ok=True)

# Load sentence transformer
sentence_transformer = SentenceTransformer('all-MiniLM-L6-v2')

# Save DataLoaders
with open(os.path.join(save_path, 'qa_train_loader.pkl'), 'wb') as f:
    pickle.dump(qa_train_loader, f)
with open(os.path.join(save_path, 'qa_val_loader.pkl'), 'wb') as f:
    pickle.dump(qa_val_loader, f)
with open(os.path.join(save_path, 'triple_train_loader.pkl'), 'wb') as f:
    pickle.dump(triple_train_loader, f)

# Save triple_candidates
with open(os.path.join(save_path, 'triple_candidates.pkl'), 'wb') as f:
    pickle.dump(triple_candidates, f)

# Save T5 QA model (t5_triple_model is not used in the backup plan)
torch.save(t5_qa_model.state_dict(), os.path.join(save_path, 't5_qa_model.pt'))

# Save sentence_transformer
with open(os.path.join(save_path, 'sentence_transformer.pkl'), 'wb') as f:
    pickle.dump(sentence_transformer, f)

# Compute all_candidates directly from DataLoaders
all_candidates = []
for batch in qa_train_loader:
    all_candidates.extend(batch["answer"])
for batch in qa_val_loader:
    all_candidates.extend(batch["answer"])
all_candidates.extend(triple_candidates)
all_candidates = list(set(all_candidates))[:5000]

# Save all_candidates
with open(os.path.join(save_path, 'all_candidates.pkl'), 'wb') as f:
    pickle.dump(all_candidates, f)

print("Artifacts saved to Google Drive!")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Artifacts saved to Google Drive!


In [ ]:
# Fine-Tune DPR for Discriminative Retrieval

# Clear GPU memory before starting
import torch
import gc
torch.cuda.empty_cache()
gc.collect()

# Mount Google Drive
from google.colab import drive
drive.mount('/content/drive')

# Load artifacts
import pickle
import os
from transformers import DPRContextEncoder, DPRQuestionEncoder, DPRContextEncoderTokenizer, DPRQuestionEncoderTokenizer
from torch.utils.data import DataLoader
import numpy as np
from tqdm import tqdm

save_path = '/content/drive/MyDrive/bert_retrieval_artifacts'

# Load DataLoaders
with open(os.path.join(save_path, 'qa_train_loader.pkl'), 'rb') as f:
    qa_train_loader = pickle.load(f)
with open(os.path.join(save_path, 'qa_val_loader.pkl'), 'rb') as f:
    qa_val_loader = pickle.load(f)

# Load all_candidates
with open(os.path.join(save_path, 'all_candidates.pkl'), 'rb') as f:
    all_candidates = pickle.load(f)

# Load DPR models and tokenizers
ctx_encoder = DPRContextEncoder.from_pretrained(CONFIG.DPR_CTX_MODEL_NAME).to(CONFIG.DEVICE)
question_encoder = DPRQuestionEncoder.from_pretrained(CONFIG.DPR_QUESTION_MODEL_NAME).to(CONFIG.DEVICE)
ctx_tokenizer = DPRContextEncoderTokenizer.from_pretrained(CONFIG.DPR_CTX_MODEL_NAME)
question_tokenizer = DPRQuestionEncoderTokenizer.from_pretrained(CONFIG.DPR_QUESTION_MODEL_NAME)

# Encode all candidates
print("Encoding candidates...")
candidate_inputs = ctx_tokenizer(all_candidates, return_tensors="pt", padding=True, truncation=True, max_length=CONFIG.MAX_LENGTH)
candidate_inputs = {k: v.to(CONFIG.DEVICE) for k, v in candidate_inputs.items()}
with torch.no_grad():
    candidate_embeddings = ctx_encoder(**candidate_inputs).pooler_output
torch.save(candidate_embeddings, os.path.join(save_path, 'dpr_candidate_embeddings.pt'))

# Fine-tune DPR
optimizer = torch.optim.AdamW(list(ctx_encoder.parameters()) + list(question_encoder.parameters()), lr=2e-5)
scaler = torch.cuda.amp.GradScaler()
epochs = CONFIG.MAX_EPOCHS

for epoch in range(epochs):
    ctx_encoder.train()
    question_encoder.train()
    total_loss = 0
    for batch in tqdm(qa_train_loader, desc=f"Epoch {epoch+1}/{epochs}"):
        question_inputs = {
            "input_ids": batch["dpr_input_ids"].to(CONFIG.DEVICE),
            "attention_mask": batch["dpr_attention_mask"].to(CONFIG.DEVICE)
        }
        correct_answers = batch["answer"]
        optimizer.zero_grad(set_to_none=True)
        with torch.cuda.amp.autocast():
            question_embeddings = question_encoder(**question_inputs).pooler_output  # Shape: (batch_size, 768)
            similarities = torch.matmul(question_embeddings, candidate_embeddings.T)  # Shape: (batch_size, num_candidates)
            batch_size = question_embeddings.size(0)
            labels = torch.zeros(batch_size, dtype=torch.long, device=CONFIG.DEVICE)
            for i in range(batch_size):
                correct_answer = correct_answers[i]
                if correct_answer in all_candidates:
                    labels[i] = all_candidates.index(correct_answer)
                else:
                    labels[i] = 0  # Fallback to first candidate if not found
            loss = torch.nn.functional.cross_entropy(similarities, labels)
        scaler.scale(loss).backward()
        scaler.step(optimizer)
        scaler.update()
        total_loss += loss.item()
        del question_inputs, similarities, labels, loss
        torch.cuda.empty_cache()
    avg_loss = total_loss / len(qa_train_loader)
    print(f"Epoch {epoch+1}/{epochs} - Train Loss: {avg_loss:.4f}")

# Evaluate DPR
def evaluate_dpr(ctx_encoder, question_encoder, val_loader, candidates, small_candidate_pool: bool = False):
    ctx_encoder.eval()
    question_encoder.eval()
    mrr, precision_at_1 = [], []
    eval_candidates = candidates[:100] if small_candidate_pool else candidates
    print(f"Using candidate pool size: {len(eval_candidates)}")
    candidate_inputs = ctx_tokenizer(eval_candidates, return_tensors="pt", padding=True, truncation=True, max_length=CONFIG.MAX_LENGTH)
    candidate_inputs = {k: v.to(CONFIG.DEVICE) for k, v in candidate_inputs.items()}
    with torch.no_grad():
        candidate_embeddings = ctx_encoder(**candidate_inputs).pooler_output
    with torch.no_grad():
        for batch in tqdm(val_loader, desc="Evaluating"):
            question_inputs = {
                "input_ids": batch["dpr_input_ids"].to(CONFIG.DEVICE),
                "attention_mask": batch["dpr_attention_mask"].to(CONFIG.DEVICE)
            }
            references = batch["answer"]
            question_embeddings = question_encoder(**question_inputs).pooler_output
            similarities = torch.matmul(question_embeddings, candidate_embeddings.T)
            rankings = torch.argsort(similarities, dim=1, descending=True)
            for i, (ranking, ref) in enumerate(zip(rankings, references)):
                ref_idx = eval_candidates.index(ref) if ref in eval_candidates else -1
                if ref_idx == -1:
                    continue
                rank = (ranking == ref_idx).nonzero(as_tuple=True)[0].item() + 1 if ref_idx in ranking else len(eval_candidates)
                mrr.append(1.0 / rank)
                precision_at_1.append(1.0 if rank == 1 else 0.0)
            del question_inputs, similarities, rankings
            torch.cuda.empty_cache()
    avg_mrr = np.mean(mrr)
    avg_precision_at_1 = np.mean(precision_at_1)
    print("DPR Evaluation:")
    print(f"MRR: {avg_mrr:.4f}")
    print(f"Precision@1: {avg_precision_at_1:.4f}")
    return avg_mrr, avg_precision_at_1

# Evaluate DPR on full and small candidate pools
dpr_mrr_full, dpr_precision_full = evaluate_dpr(ctx_encoder, question_encoder, qa_val_loader, all_candidates, small_candidate_pool=False)
dpr_mrr_small, dpr_precision_small = evaluate_dpr(ctx_encoder, question_encoder, qa_val_loader, all_candidates, small_candidate_pool=True)

# Save DPR models
ctx_encoder.save_pretrained(os.path.join(CONFIG.BASE_PATH, "dpr_ctx_encoder_final"))
question_encoder.save_pretrained(os.path.join(CONFIG.BASE_PATH, "dpr_question_encoder_final"))
print("Saved DPR models.")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


config.json:   0%|          | 0.00/492 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/438M [00:00<?, ?B/s]

Some weights of the model checkpoint at facebook/dpr-ctx_encoder-single-nq-base were not used when initializing DPRContextEncoder: ['ctx_encoder.bert_model.pooler.dense.bias', 'ctx_encoder.bert_model.pooler.dense.weight']
- This IS expected if you are initializing DPRContextEncoder from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DPRContextEncoder from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


pytorch_model.bin:   0%|          | 0.00/438M [00:00<?, ?B/s]

...

Epoch 3/3 - Train Loss: 3.1710
Using candidate pool size: 2024


Evaluating: 100%|██████████| 7/7 [00:01<00:00,  5.47it/s]


DPR Evaluation:
MRR: 0.4157
Precision@1: 0.3200
Using candidate pool size: 100


Evaluating: 100%|██████████| 7/7 [00:00<00:00, 11.88it/s]


DPR Evaluation:
MRR: 0.5214
Precision@1: 0.3750
Saved DPR models.


In [ ]:
# Fine-Tune T5 QA and Evaluate Ensemble with DPR

# Clear GPU memory before starting
import torch
import gc
torch.cuda.empty_cache()
gc.collect()

# Mount Google Drive
from google.colab import drive
drive.mount('/content/drive')

# Load artifacts
import pickle
import os
from transformers import T5ForConditionalGeneration, T5Tokenizer, DPRContextEncoder, DPRQuestionEncoder, DPRContextEncoderTokenizer, DPRQuestionEncoderTokenizer
from torch.utils.data import DataLoader
import numpy as np
from tqdm import tqdm
from nltk.translate.bleu_score import sentence_bleu
from rouge_score import rouge_scorer
import bert_score

save_path = '/content/drive/MyDrive/bert_retrieval_artifacts'

# Load DataLoaders
with open(os.path.join(save_path, 'qa_train_loader.pkl'), 'rb') as f:
    qa_train_loader = pickle.load(f)
with open(os.path.join(save_path, 'qa_val_loader.pkl'), 'rb') as f:
    qa_val_loader = pickle.load(f)

# Load all_candidates
with open(os.path.join(save_path, 'all_candidates.pkl'), 'rb') as f:
    all_candidates = pickle.load(f)

# Load T5 model and tokenizer
t5_qa_model = T5ForConditionalGeneration.from_pretrained(CONFIG.T5_MODEL_NAME).to(CONFIG.DEVICE)
t5_qa_model.load_state_dict(torch.load(os.path.join(save_path, 't5_qa_model.pt')))
t5_tokenizer = T5Tokenizer.from_pretrained(CONFIG.T5_MODEL_NAME)

# Load DPR models and tokenizers
ctx_encoder = DPRContextEncoder.from_pretrained(os.path.join(CONFIG.BASE_PATH, "dpr_ctx_encoder_final")).to(CONFIG.DEVICE)
question_encoder = DPRQuestionEncoder.from_pretrained(os.path.join(CONFIG.BASE_PATH, "dpr_question_encoder_final")).to(CONFIG.DEVICE)
ctx_tokenizer = DPRContextEncoderTokenizer.from_pretrained(CONFIG.DPR_CTX_MODEL_NAME)
question_tokenizer = DPRQuestionEncoderTokenizer.from_pretrained(CONFIG.DPR_QUESTION_MODEL_NAME)
candidate_embeddings = torch.load(os.path.join(save_path, 'dpr_candidate_embeddings.pt')).to(CONFIG.DEVICE)

# Fine-tune T5 QA for 1 more epoch with focus on exact answer generation
optimizer = torch.optim.AdamW(t5_qa_model.parameters(), lr=3e-4)
scaler = torch.cuda.amp.GradScaler()
epochs = 1

for epoch in range(epochs):
    t5_qa_model.train()
    total_loss = 0
    for batch in tqdm(qa_train_loader, desc=f"Epoch {epoch+1}/{epochs}"):
        input_ids = batch["t5_input_ids"].to(CONFIG.DEVICE)
        attention_mask = batch["t5_attention_mask"].to(CONFIG.DEVICE)
        labels = batch["t5_labels"].to(CONFIG.DEVICE)
        optimizer.zero_grad()
        with torch.cuda.amp.autocast():
            outputs = t5_qa_model(input_ids=input_ids, attention_mask=attention_mask, labels=labels)
            loss = outputs.loss
        scaler.scale(loss).backward()
        scaler.step(optimizer)
        scaler.update()
        total_loss += loss.item()
        del input_ids, attention_mask, labels, outputs, loss
        torch.cuda.empty_cache()
    avg_loss = total_loss / len(qa_train_loader)
    print(f"Epoch {epoch+1}/{epochs} - Train Loss: {avg_loss:.4f}")

# Evaluate T5 QA
def evaluate_t5(model, val_loader, task: str = "qa"):
    print(f"Evaluating T5 for {task}...")
    model.eval()
    bleu_scores, rouge_scores, bert_scores = [], [], []
    sample_outputs = []
    scorer = rouge_scorer.RougeScorer(['rougeL'], use_stemmer=True)

    with torch.no_grad():
        for batch in tqdm(val_loader, desc="Evaluating"):
            input_ids = batch["t5_input_ids"].to(CONFIG.DEVICE)
            attention_mask = batch["t5_attention_mask"].to(CONFIG.DEVICE)
            references = batch["answer"]
            generated_ids = model.generate(
                input_ids=input_ids,
                attention_mask=attention_mask,
                max_new_tokens=100,
                num_beams=20,
                temperature=0.5,
                no_repeat_ngram_size=2
            )
            generated_texts = [t5_tokenizer.decode(g_ids, skip_special_tokens=True) for g_ids in generated_ids]
            for gen, ref in zip(generated_texts, references):
                gen = normalize_text(gen)
                ref = normalize_text(ref)
                bleu = sentence_bleu([ref.split()], gen.split())
                rouge = scorer.score(ref, gen)['rougeL'].fmeasure
                bert_f1 = bert_score.score([gen], [ref], lang="en", verbose=False)[2].mean().item()
                bleu_scores.append(bleu)
                rouge_scores.append(rouge)
                bert_scores.append(bert_f1)
                sample_outputs.append((gen, ref))
            del input_ids, attention_mask, generated_ids
            torch.cuda.empty_cache()
    avg_bleu = np.mean(bleu_scores)
    avg_rouge = np.mean(rouge_scores)
    avg_bert = np.mean(bert_scores)
    print(f"T5 {task} Evaluation:")
    print(f"Average BLEU: {avg_bleu:.4f}")
    print(f"Average ROUGE-L: {avg_rouge:.4f}")
    print(f"Average BERTScore F1: {avg_bert:.4f}")
    print("Sample Outputs (First 5):")
    for gen, ref in sample_outputs[:5]:
        print(f"Generated: {gen}")
        print(f"Reference: {ref}\n")
    return avg_bleu, avg_rouge, avg_bert

# Evaluate T5 QA
t5_qa_bleu, t5_qa_rouge, t5_qa_bert = evaluate_t5(t5_qa_model, qa_val_loader, task="qa")

# Ensemble: T5 generates candidates, DPR re-ranks
def ensemble_evaluate(t5_model, ctx_encoder, question_encoder, val_loader, candidates, top_k: int = 30):
    print("Evaluating ensemble (T5 + DPR)...")
    t5_model.eval()
    ctx_encoder.eval()
    question_encoder.eval()
    mrr, precision_at_1 = [], []
    candidate_inputs = ctx_tokenizer(candidates, return_tensors="pt", padding=True, truncation=True, max_length=CONFIG.MAX_LENGTH)
    candidate_inputs = {k: v.to(CONFIG.DEVICE) for k, v in candidate_inputs.items()}
    with torch.no_grad():
        candidate_embeddings = ctx_encoder(**candidate_inputs).pooler_output
    with torch.no_grad():
        for batch in tqdm(val_loader, desc="Ensemble Evaluating"):
            input_ids = batch["t5_input_ids"].to(CONFIG.DEVICE)
            attention_mask = batch["t5_attention_mask"].to(CONFIG.DEVICE)
            references = batch["answer"]
            generated_ids = t5_model.generate(
                input_ids=input_ids,
                attention_mask=attention_mask,
                max_new_tokens=100,
                num_beams=30,
                temperature=0.5,
                num_return_sequences=top_k
            )
            generated_texts = [t5_tokenizer.decode(g_ids, skip_special_tokens=True).lower().strip() for g_ids in generated_ids]
            generated_texts = [generated_texts[i:i+top_k] for i in range(0, len(generated_texts), top_k)]
            for i, (gen_texts, ref) in enumerate(zip(generated_texts, references)):
                if not gen_texts:  # Skip if no candidates generated
                    continue
                gen_inputs = question_tokenizer(gen_texts, return_tensors="pt", padding=True, truncation=True, max_length=CONFIG.MAX_LENGTH)
                gen_inputs = {k: v.to(CONFIG.DEVICE) for k, v in gen_inputs.items()}
                gen_embeddings = question_encoder(**gen_inputs).pooler_output
                gen_similarities = torch.matmul(gen_embeddings, candidate_embeddings.T)
                rankings = torch.argsort(gen_similarities, dim=1, descending=True)
                if rankings.size(0) == 0 or rankings.size(1) == 0:  # Skip if rankings are empty
                    continue
                top_idx = rankings[0, 0].item()  # Take the top-ranked candidate
                if top_idx >= len(gen_texts):  # Ensure top_idx is within bounds
                    continue
                predicted = gen_texts[top_idx]
                ref_idx = gen_texts.index(ref) if ref in gen_texts else -1
                if ref_idx == -1:
                    continue
                rank = (rankings[:, 0] == ref_idx).nonzero(as_tuple=True)[0].item() + 1 if ref_idx in rankings[:, 0] else len(gen_texts)
                mrr.append(1.0 / rank)
                precision_at_1.append(1.0 if rank == 1 else 0.0)
            del input_ids, attention_mask, generated_ids, gen_inputs, gen_similarities, rankings
            torch.cuda.empty_cache()
    avg_mrr = np.mean(mrr) if mrr else 0.0
    avg_precision_at_1 = np.mean(precision_at_1) if precision_at_1 else 0.0
    print("Ensemble (T5 + DPR) Evaluation:")
    print(f"MRR: {avg_mrr:.4f}")
    print(f"Precision@1: {avg_precision_at_1:.4f}")
    return avg_mrr, avg_precision_at_1

# Evaluate ensemble
ensemble_mrr, ensemble_precision = ensemble_evaluate(t5_qa_model, ctx_encoder, question_encoder, qa_val_loader, all_candidates)

# Save evaluation results
results = {
    "t5_qa": {"bleu": t5_qa_bleu, "rouge": t5_qa_rouge, "bertscore": t5_qa_bert},
    "dpr_full": {"mrr": dpr_mrr_full, "precision_at_1": dpr_precision_full},
    "dpr_small_pool": {"mrr": dpr_mrr_small, "precision_at_1": dpr_precision_small},
    "ensemble": {"mrr": ensemble_mrr, "precision_at_1": ensemble_precision}
}

import json
results_path = os.path.join(CONFIG.BASE_PATH, "evaluation_results.json")
with open(results_path, "w") as f:
    json.dump(results, f)
print(f"Saved evaluation results at {results_path}")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'DPRQuestionEncoderTokenizer'. 
The class this function is called from is 'DPRContextEncoderTokenizer'.
Epoch 1/1: 100%|██████████| 88/88 [00:09<00:00,  9.07it/s]


Epoch 1/1 - Train Loss: 1.1526
Evaluating T5 for qa...


Evaluating:   0%|          | 0/7 [00:00<?, ?it/s]/usr/local/lib/python3.11/dist-packages/transformers/generation/configuration_utils.py:515: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.5` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
Some weights of RobertaModel were not initialized from the model checkpoint at roberta-large and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Some weights of RobertaModel were not initialized from the model checkpoint at roberta-large and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Some weights of RobertaModel were not initialized from the 

T5 qa Evaluation:
Average BLEU: 0.0049
Average ROUGE-L: 0.0401
Average BERTScore F1: 0.7827
Sample Outputs (First 5):
Generated: tidal etts syria london university of new york university university college of arts and sciences université de saintlaurent university school of anthropologie interdisciplinary anthropology and linguistics
Reference: 80

Generated: ossasia etio ayota
Reference: unanswerable

Generated: symphonie ettas teesa adosso
Reference: external senses

Generated: oxford and cambridge admit more eonians each year than applicants from whole country who qualify for free school meals in april 2011 labour mp david lammy described as unfair and indefensible fact that ford university had organised nineoutach events at on in 2010
Reference: wellington college

Generated: symphonie ossas ettees
Reference: 200 million kilowatt hours

Evaluating ensemble (T5 + DPR)...


Ensemble Evaluating: 100%|██████████| 7/7 [01:07<00:00,  9.68s/it]

Ensemble (T5 + DPR) Evaluation:
MRR: 0.0000
Precision@1: 0.0000
Saved evaluation results at /content/drive/MyDrive/LJMU-Datasets/evaluation_results.json
